Использование обученных моделей для:

1) Классификации текстов

2) Поиск похожих текстов на слова

# Готовим DataSet

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/saurabh3949/Text-Classification-Datasets/master/TREC.train.all'
df = pd.read_csv(url, sep='\t', header=None, names=['Label'],encoding='ISO-8859-1')

In [ ]:
df['Label'] = df['Label'].apply(lambda x: x.split(" "))

In [ ]:
classes = df['Label'].apply(lambda x: x[0])
df['Classes'] = classes

In [ ]:
df['Text'] = df['Label'].apply(lambda x: x[1:])

In [ ]:
df = df.drop("Label", axis = 1)

In [ ]:
df['Text'] = df['Text'].apply(lambda x: " ".join(x))

In [ ]:
df = df.loc[:,["Text","Classes"]]

In [ ]:
df_mini = df.iloc[:2000]

# Скачиваем модель

In [ ]:
import transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

# Тестируем модель

In [ ]:
text = "Replace me by any text you'd like."

In [ ]:
encoded_input = tokenizer(text, return_tensors='pt')

In [ ]:
encoded_input

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
output = model(**encoded_input)

In [ ]:
output[1].squeeze(0)

tensor([-1.0000,  0.1268, -0.9992, -0.4620, -0.9979, -0.8092, -0.9974,  0.5125,
         0.0925, -0.1791, -0.3463, -0.0690, -0.1103,  1.0000, -0.8554, -0.9489,
         0.8608, -0.0280, -0.9388,  0.9993,  0.9701,  0.0961,  0.9991,  0.8984,
        -1.0000, -0.0017, -0.9983,  0.8225,  0.9993, -0.0302, -0.1821, -0.0231,
        -0.9867, -0.7341,  0.8197,  0.9998, -0.9358, -0.1245,  0.9200, -0.9990,
         0.9792,  0.9913, -0.9997,  0.9827, -0.9999, -0.2473, -0.9997,  0.9990,
         0.9927,  0.9964,  0.8801, -0.6949, -0.0670,  0.8253,  0.9987,  0.9969,
        -0.9286, -0.8519,  0.9287, -0.3063, -0.1007,  0.4718, -0.9725,  0.9945,
        -0.8136, -1.0000,  0.2504,  0.9211,  0.9167,  0.9917,  0.9856,  0.1915,
        -0.9998, -0.0468,  0.6239, -0.9931, -0.8944,  0.3323, -0.6123,  0.1455,
         0.7395, -0.0462, -0.9951, -1.0000,  0.9997, -0.9338,  0.6063,  0.9734,
         0.6361,  0.9630, -0.7342,  0.9905, -0.9952,  0.9931, -0.7291,  0.3992,
        -0.9362,  0.8055, -0.9999, -0.90

In [ ]:
import torch
@torch.inference_mode()
def get_vectors(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  output = output[1].squeeze(0)
  return output

# Предобрабываем df

In [ ]:
df_mini['Vector'] = df_mini['Text'].apply(lambda x: get_vectors(x).numpy())

# Проверим, что похожие классы текстов находятся рядом

In [ ]:
vectorized_df = pd.DataFrame(df_mini['Vector'].values.tolist())

In [ ]:
df_mini = pd.concat([df_mini,vectorized_df], axis = 1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors = 5, weights = 'distance')

In [ ]:
knn_model.fit(vectorized_df,df_mini['Classes'])

KNeighborsClassifier(weights='distance')

In [ ]:
sample = vectorized_df.iloc[14].values.reshape(1,-1)

In [ ]:
dist, neights = knn_model.kneighbors(X = sample,n_neighbors = 10,)

In [ ]:
dist

array([[1.68587394e-07, 1.94825020e+00, 2.04396934e+00, 2.29677670e+00,
        2.42680546e+00, 2.47501843e+00, 2.48206678e+00, 2.52564580e+00,
        2.52796648e+00, 2.53179351e+00]])

In [ ]:
neights

array([[  14,  639, 1400,  212,  780, 1179,  418, 1932,  380, 1242]])

In [ ]:
df_mini['Text'].iloc[14]

'What is considered the costliest disaster the insurance industry has ever faced ?'

In [ ]:
df_mini.loc[neights[0],:0]

,Text,Classes,Vector,0
14,What is considered the costliest disaster the ...,1,"[-0.9999519, 0.12445439, -0.9993391, 0.9135453...",-0.999952
639,What day of the week sees the most fatal car a...,5,"[-0.9999501, 0.02241693, -0.9982628, 0.8430316...",-0.999950
1400,What city is often called The Insurance Capita...,4,"[-0.9999635, 0.099084005, -0.99963427, 0.96015...",-0.999964
212,What South American city has the world 's high...,4,"[-0.99985224, 0.053765573, -0.99742967, 0.9806...",-0.999852
780,What Asian country has a bill of rights for co...,4,"[-0.99998766, 0.13625109, -0.9993325, 0.959167...",-0.999988
1179,Where can I find a list of all the companies i...,4,"[-0.9999839, 0.050414868, -0.99932706, 0.91171...",-0.999984
418,What Nevada center has been dubbed The Biggest...,4,"[-0.9999876, 0.08403866, -0.99987173, 0.993275...",-0.999988
1932,What are the biggest Indian airports ?,4,"[-0.9999854, 0.033322256, -0.9995017, 0.787485...",-0.999985
380,Who is the only prime minister of Canada to se...,3,"[-0.99999726, 0.20408824, -0.9992403, 0.663108...",-0.999997
1242,What are the largest breweries in the world ?,4,"[-0.99999154, -0.011155554, -0.99971026, 0.959...",-0.999992


# Начинаем эксперементировать с порогом Евклидового расстояния

По моим личным наблюдениям - порог меньше 1 дает ужасных совершенно не похожих по содержанию соседей

Для каждого текста вернем N соседей порог которых меньше 1

In [ ]:
import numpy as np
pd.set_option('display.max_colwidth', 30)

In [ ]:
def get_good_neighbours(idx,threshold:float):
  sample = vectorized_df.iloc[idx].values.reshape(1,-1)
  dist, neights = knn_model.kneighbors(X = sample,n_neighbors = 10)
  dist = dist[0][1:]
  neights = neights[0][1:]
  if np.where(dist<threshold)[0].size == 0:
    return np.nan
  else:
    result = neights[np.where(dist<threshold)[0]]
    return result

leng = len(df_mini)
df_mini["Neigs"] = 0

N = []
for idx in range(leng):
  N.append(get_good_neighbours(idx,0.9))

df_mini["Neigs"] = N

In [ ]:
mask = ~df_mini['Neigs'].isna()
print(f"Длина {len(df_mini[mask])}")

Длина 59


In [ ]:
# Получили DataFrame у которого есть какие либо соседи
valuation_df = df_mini[mask]

In [ ]:
# For each element in text

def see_good_texts(valuation_df: pd.DataFrame):

  big_box = []
  max_similiar_texts = []
  for idx,value  in zip(valuation_df['Neigs'].index,valuation_df['Neigs'].values):
    a = valuation_df["Text"][idx]

    small_box = []
    for similar_text_idx in value:
      small_box+= [valuation_df["Text"][similar_text_idx]]

    chunk = [[a,*small_box]]

    chunk_leng = len(chunk[0])
    max_similiar_texts.append(chunk_leng)

    big_box += chunk

  max_similiar_texts = max(max_similiar_texts)

  return pd.DataFrame(data = big_box, columns = ["Orig"]+[f"sample{i}" for i in range(1,max_similiar_texts)], index = valuation_df['Neigs'].index)

In [ ]:
comparison_df = see_good_texts(valuation_df)

In [ ]:
pd.set_option('display.max_colwidth', None)
comparison_df

,Orig,sample1,sample2,sample3,sample4
53,Where did the term `` 86ed '' come from ?,Where did the term `` 86 '' come from ?,None,None,None
94,What is `` dew point '' ?,What color is `` ash '' ?,What is `` snoogans '' ?,None,None
101,What is the origin of the word `` attic '' ?,What is the origin of the word `` mind '' ?,None,None,None
119,What does the Georgetown `` Hoya '' mean ?,What does the name `` Justin '' mean ?,What does the name `` Anthony '' mean ?,None,None
121,Which two products use a tiger as their symbol ?,Which two products use a tiger as their symbol ?,None,None,None
216,What is `` snoogans '' ?,What is `` dew point '' ?,None,None,None
325,"What is the latitude and longitude of El Paso , Texas ?","What is the latitude and longitude of El Paso , Texas ?",None,None,None
351,What is the speed of the Mississippi River ?,What is the speed of the Mississippi River ?,What is the speed of the Mississippi River ?,None,None
369,What is the speed of the Mississippi River ?,What is the speed of the Mississippi River ?,What is the speed of the Mississippi River ?,None,None
443,What does G.M.T. stand for ?,What does e.g. stand for ?,None,None,None


# Перейдем к классификации текстов

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/saurabh3949/Text-Classification-Datasets/master/TREC.train.all'
df = pd.read_csv(url, sep='\t', header=None, names=['Label'],encoding='ISO-8859-1')

In [ ]:
df['Label'] = df['Label'].apply(lambda x: x.split(" "))

In [ ]:
classes = df['Label'].apply(lambda x: x[0])
df['Classes'] = classes

In [ ]:
df['Text'] = df['Label'].apply(lambda x: x[1:])

In [ ]:
df = df.drop("Label", axis = 1)

In [ ]:
df['Text'] = df['Text'].apply(lambda x: " ".join(x))

In [ ]:
df = df.loc[:,["Text","Classes"]]

In [ ]:
import torch
from sklearn.metrics import classification_report

In [ ]:
import torch
@torch.inference_mode()
def get_vectors(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  output = output[1].squeeze(0)
  output = output.numpy()
  return output

In [ ]:
df["Vector"] = df["Text"].apply(lambda x: get_vectors(x))


In [ ]:
pd.set_option('display.max_colwidth', 20)

In [ ]:
vectorized_df = pd.DataFrame(df['Vector'].values.tolist())

In [ ]:
df = pd.concat([df,vectorized_df], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.head(2)

,Text,Classes,Vector,0,1,2,3,4,5,6,...,118,119,120,121,122,123,124,125,126,127
0,How did serfdom ...,0,"[-0.99998635, 0....",-0.999986,0.121004,-0.999302,0.112896,-0.999741,-0.050388,-0.997083,...,0.189968,-0.981982,0.307353,-0.999982,-0.640556,0.849185,-0.999947,0.932246,0.979752,0.949779
1,What films featu...,1,"[-0.9999781, 0.0...",-0.999978,0.077666,-0.999316,0.973250,-0.999605,0.435007,-0.995725,...,0.117527,-0.993199,0.311935,-0.999999,-0.903571,0.902569,-0.999798,0.998968,0.984857,0.816294


In [ ]:
X = df.iloc[:,2:]
Y = df["Classes"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.drop("Vector", axis = 1)
X_test = X_test.drop("Vector", axis = 1)

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
catboost_model = CatBoostClassifier(verbose= False)
catboost_model.fit(X_train,y_train)

In [ ]:
prediction = catboost_model.predict(X_test)

In [ ]:
# So so results :c
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.61      0.57      0.59       227
           1       0.56      0.60      0.58       246
           2       1.00      0.43      0.61        23
           3       0.72      0.82      0.77       240
           4       0.73      0.69      0.71       166
           5       0.62      0.58      0.60       189

    accuracy                           0.65      1091
   macro avg       0.71      0.62      0.64      1091
weighted avg       0.65      0.65      0.65      1091



# Попробую использовать нейронную сеть на нашем датасете, чтобы она давала более хороший результат

In [ ]:
input = tokenizer(df['Text'][0], return_tensors = "pt")
model_output = model(**input)['pooler_output']
print(model_output.size())

torch.Size([1, 128])


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
tiny_bert = AutoModel.from_pretrained("prajjwal1/bert-tiny")

In [ ]:
import torch
from torch import nn
from torch.optim import Adam

# Must create dataloader and apply tokenizer firstly

class BertClassification(nn.Module):

  def __init__(self,bert_model,num_classes):
    super().__init__()
    self.bert = bert_model #output of size [1,128]
    self.fc = nn.Sequential(nn.Linear(128,20),
                            nn.ReLU(),
                            nn.Linear(20,num_classes),
                            nn.Sigmoid())

  def get_vectors(self,x:dict):
    vectors = self.bert(**x)['pooler_output'][0]
    return vectors

  def forward(self,x):
    ouput = self.fc(x) # must be tensor of size [6]
    return output

In [ ]:
class BertClassification_V2(nn.Module):

  def __init__(self,bert_model,num_classes):
    super().__init__()
    self.bert = bert_model #output of size [1,128]
    self.fc = nn.Sequential(nn.Linear(128,20),
                            nn.ReLU(),
                            nn.Linear(20,num_classes),
                            nn.Sigmoid())

  def get_vectors(self,x:dict):
    vectors = self.bert(**x)['pooler_output'][0]
    return vectors

  def forward(self,x):
    output = self.get_vectors(x)
    output = self.fc(output) # must be tensor of size [6]
    return output

In [ ]:
model = BertClassification_V2(tiny_bert,6)
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertClassification_V2(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwi

In [ ]:
# loss_fn, optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

# Создадим DataLoader

In [ ]:
len(train_dataset)

4906

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X = df['Text']
Y = df['Classes']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.1)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
def tokenize_text(text:str):
  tokenized_dict = tokenizer(text, padding = True, return_tensors = "pt").to(device)

  #tokenized_dict = torch.tensor(tokenized_dict)

  return tokenized_dict

In [ ]:
class Custom_DataSet(Dataset):

    def __init__(self, x_train,y_train,transform = None):
        self.X = x_train.values
        self.Y = torch.tensor(y_train.values,dtype = torch.float32)
        self.transform = tokenize_text

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, item):
      res = self.X[item], self.Y[item]
      return self.transform(self.X[item]), self.Y[item]

In [ ]:
train_dataset = Custom_DataSet(X_train,y_train,transform = tokenize_text)
test_dataset = Custom_DataSet(X_test,y_test,transform = tokenize_text)

In [ ]:
# Valuation a qualtiy of our train_datas
# Batch is a list of lists[a,b]

# def see_batch():
#   for batch in train_loader:
#     return batch


def collate_fn(batch):
  batch = np.array(batch, dtype = "object")

  # working with Y
  y = torch.tensor([int(i.item()) for i in batch[:,1]]).to(torch.long)
  #y = y.reshape(-1,1)

  # working with X
  x = batch[:,0].tolist()

  return x,y

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=130, shuffle=True, num_workers=4, pin_memory=True, collate_fn = collate_fn)
test_loader = DataLoader(test_dataset, batch_size=130, shuffle=False, num_workers=4, pin_memory=True, collate_fn = collate_fn)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Функции тренировки и evaluation

In [ ]:
from tqdm import tqdm

In [ ]:
def get_big_x(x):
  big_x = None
  for tokenized_text in x:
    output = model.get_vectors(tokenized_text)
    if big_x is None:
      big_x = output.reshape(1,-1)
    else:
        output = output.reshape(1,-1)
        big_x = torch.cat((big_x,output), dim = 0)

  big_x = big_x.to(torch.float32)

  return big_x

In [ ]:
def train(model):
  model.train()
  total_loss = 0

  for x,y in tqdm(train_loader, desc = "Training"):
    optimizer.zero_grad()

    # x = x.to(device)
    # y = y.to(device)

    big_x = get_big_x(x)

    #big_x = big_x.unsqueeze(1)

    output = model.fc(big_x) # must be tensor fo size 6 # ALLERT! FORWARD IS NOT WORKING

    #return y,output

    loss = loss_fn(output,y)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  total_loss = total_loss/len(train_loader)
  return total_loss

In [ ]:
for e in range(10):
  ll = train(model)
  print(ll)

Training:   0%|          | 0/38 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Training: 100%|██████████| 38/38 [01:38<00:00,  2.60s/it]


1.186347992796647


Training:   0%|          | 0/38 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
x_s,y_s = next(iter(test_loader))

In [ ]:
prediction = torch.argmax(model.fc(get_big_x(x_s)), dim = 1)

In [ ]:
sum(y_s == prediction)

tensor(109)

In [ ]:
y_s

tensor([5, 4, 5, 3, 5, 1, 5, 5, 3, 1, 4, 4, 5, 1, 3, 0, 4, 4, 0, 3, 3, 4, 3, 4,
        4, 3, 1, 5, 3, 3, 5, 3, 0, 5, 4, 1, 0, 0, 1, 1, 4, 3, 0, 3, 0, 0, 1, 4,
        1, 0, 4, 4, 1, 1, 3, 0, 1, 5, 3, 1, 1, 1, 0, 3, 0, 5, 3, 3, 3, 4, 3, 1,
        4, 1, 1, 1, 0, 5, 4, 1, 5, 0, 1, 1, 1, 3, 5, 5, 3, 5, 5, 5, 0, 0, 0, 3,
        3, 1, 2, 1, 3, 1, 4, 1, 5, 0, 5, 1, 1, 4, 3, 1, 3, 1, 3, 2, 5, 5, 0, 1,
        1, 0, 1, 5, 1, 4, 1, 3, 3, 0])

In [ ]:
prediction

tensor([5, 3, 3, 1, 3, 4, 0, 5, 1, 5, 3, 5, 3, 0, 4, 0, 3, 3, 5, 5, 1, 1, 3, 4,
        1, 3, 5, 4, 3, 0, 1, 0, 4, 3, 5, 4, 3, 5, 5, 1, 0, 1, 1, 5, 4, 5, 1, 1,
        4, 1, 5, 0, 5, 0, 5, 0, 0, 5, 0, 4, 5, 4, 1, 4, 4, 4, 3, 4, 0, 1, 0, 0,
        3, 5, 0, 4, 5, 4, 4, 3, 0, 4, 3, 5, 0, 0, 1, 1, 0, 5, 4, 3, 1, 4, 3, 1,
        3, 1, 3, 5, 1, 4, 5, 3, 5, 4, 4, 0, 3, 5, 5, 0, 1, 3, 4, 3, 5, 4, 1, 3,
        0, 5, 0, 4, 0, 3, 1, 3, 5, 0])

In [ ]:
total_loss

1.5730945248352854

In [ ]:
total_loss

1.3286040550784062

In [ ]:
model.forward(get_big_x(x_s))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.6194e+00,  4.2557e-01, -3.1865e+00,  ..., -5.8355e-03,
          -9.3564e-01, -1.0759e-02],
         [-2.2656e+00,  1.1381e+00,  8.3347e-01,  ..., -1.1366e+00,
          -7.9659e-01,  1.1612e+00],
         [-1.1523e+00,  3.4688e-01,  6.0958e-01,  ..., -1.6412e+00,
           4.3674e-02, -1.8458e-01],
         ...,
         [ 2.0470e-01,  2.6800e-01,  2.3521e-03,  ..., -9.3579e-01,
          -1.1946e+00, -3.1875e-01],
         [-4.3293e-01,  5.1984e-01,  4.0717e-01,  ..., -1.3465e+00,
          -1.2996e+00, -5.9949e-01],
         [-1.7133e+00,  5.2311e-01,  4.1584e-01,  ..., -1.3871e+00,
          -9.1107e-01,  5.0250e-01]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-1.0000,  0.1268, -0.9992, -0.4620, -0.9979, -0.8092, -0.9974,  0.5125,
          0.0925, -0.1791, -0.3463, -0.0690, -0.1103,  1.0000, -0.8554, -0.9489,
          0.8608, -0.0280, -0.9388,  0.9993,  0.9701,  0.0961,  0.9991,  0.898

# Экспереминтирую с новой версией модели

In [ ]:
class BertClassification_V2(nn.Module):

  def __init__(self,bert_model,num_classes):
    super().__init__()
    self.bert = bert_model #output of size [1,128]
    self.fc = nn.Sequential(nn.Linear(128,20),
                            nn.ReLU(),
                            nn.Linear(20,num_classes),
                            nn.Sigmoid())

  def get_vectors(self,x:dict):
    vectors = self.bert(**x)['pooler_output'][0]
    return vectors

  def forward(self,x):
    output = self.get_vectors(x)
    output = self.fc(output) # must be tensor of size [6]
    return output

In [ ]:
model = BertClassification_V2(tiny_bert,6)
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertClassification_V2(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwi

In [ ]:
# loss_fn, optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
def stack_output(x):
  stacked_output = None

  for tokenized_text in x:
    output = model.forward(tokenized_text)

    if stacked_output is None:
      stacked_output = output.unsqueeze(0)
      #return stacked_output
    else:
      output = output.unsqueeze(0)
      stacked_output = torch.cat((stacked_output,output), dim = 0)

  stacked_output = stacked_output.to(torch.float32)

  return stacked_output

In [ ]:
def train(model):
  model.train()
  total_loss = 0
  total_correct = 0
  total_obj = 0

  for x,y in tqdm(train_loader, desc = "Training"):
    optimizer.zero_grad()

    # x = x.to(device)
    # y = y.to(device)

    stacked_output = stack_output(x)
    prediction = torch.argmax(stacked_output, dim = 1)

    loss = loss_fn(stacked_output,y)
    loss.backward()
    optimizer.step()

    total_obj += len(y)

    correct = sum(prediction == y).item()
    total_correct += correct

    total_loss += loss.item()

  total_loss = total_loss/len(train_loader)
  epoch_accuracy = total_correct / total_obj

  return total_loss, epoch_accuracy

In [ ]:
loss_epoch = []
accuracy_epoch = []

for epoch in range(25):
  tl,ea = train(model)
  loss_epoch += [tl]
  accuracy_epoch += [ea]

Training:   0%|          | 0/38 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Training: 100%|██████████| 38/38 [01:35<00:00,  2.52s/it]


In [ ]:
@torch.inference_mode()
def evaluation(model):
  model.eval()

  all_predictions = []
  y_true = []

  for x, y in tqdm(test_loader, desc = "Evaluation"):
    stacked_output = stack_output(x)
    prediction = torch.argmax(stacked_output, dim = 1)
    all_predictions +=  prediction.tolist()
    y_true += y.tolist()

  return all_predictions, y_true

In [ ]:
all_predictions, y_true = evaluation(model)

Evaluation: 100%|██████████| 5/5 [00:02<00:00,  1.68it/s]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true,all_predictions))

              precision    recall  f1-score   support

           0       0.82      0.80      0.81       118
           1       0.77      0.71      0.74       126
           2       1.00      0.10      0.18        10
           3       0.77      0.88      0.82       131
           4       0.89      0.89      0.89        75
           5       0.90      0.93      0.91        86

    accuracy                           0.82       546
   macro avg       0.86      0.72      0.73       546
weighted avg       0.82      0.82      0.81       546

